In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from geoscripts import gchemplots as gp

In [ ]:
# Read in organized data
data = pd.read_csv('processed/data.csv',index_col=0)

# Filter for J and mafic
ternary_total = data['Ta']*2+data['Tb']*3+data['Th']
filter_period = (data['Period']=='Jurassic')|(data['Period']=='Jurassic?')
filter_major =  (data['SiO2']<55)
filter_trace = (data['Tb']*3/ternary_total>0.5)
data_filtered = data[filter_period&filter_major]

data_gtoid = data[data['SiO2']>55]

print(data_filtered.index)

In [ ]:
Tb,Th,Ta = data_filtered[['Tb','Th','Ta']].T.values

fig = plt.figure()
ax = fig.add_subplot(111,projection='ternary')
gp.cabanis(Tb,Th,Ta)

In [ ]:
fig,axs = plt.subplots(2,figsize=(5,7))
gp.REE(data_filtered,ax=axs[0])
gp.immobile(data_filtered,ax=axs[1])

In [ ]:
# Look at specific samples

samples = ['G22076','G22081']

smp_data = data.loc[samples]

SiO2,Na2O,K2O = smp_data[['SiO2','Na2O','K2O']].T.values
gp.TASsm(SiO2,Na2O,K2O)

Tb,Th,Ta = smp_data[['Tb','Th','Ta']].T.values

fig = plt.figure()
ax = fig.add_subplot(111,projection='ternary')
gp.cabanis(Tb,Th,Ta)

fig2,axs = plt.subplots(2,figsize=(5,7))
gp.REE(smp_data,ax=axs[0])
gp.immobile(smp_data,ax=axs[1])


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='ternary')

fig2,axs = plt.subplots(2,figsize=(5,7))

fig3,ax3 = plt.subplots(1)
fig4,ax4 = plt.subplots(1)
fig5,ax5 = plt.subplots(1)

lat = pd.cut(data_filtered['Latitude'],4)
lon = pd.cut(data_filtered['Longitude'],4)
# domain = pd.cut(data_filtered['S_Domain'])

groups = data_filtered.groupby(data_filtered['S_Domain'])
for k, group in groups:
    Tb,Th,Ta = group[['Tb','Th','Ta']].T.values
    gp.cabanis(Tb,Th,Ta,ax=ax,label=k)
    gp.REE(group,ax=axs[0])
    gp.immobile(group,ax=axs[1])
    ax3.scatter(group['Longitude'],group['Latitude'])
    ax4.scatter(group['TiO2'],group['V'])
    ax5.scatter(group['Nb']/group['Yb'],group['Th']/group['Yb'])
    ax5.set_xscale('log')
    ax5.set_yscale('log')
    ax5.set_xlim(0.1,100)
    ax5.set_ylim(0.01,10)

ax.legend()

In [ ]:
smp_data = data_filtered[data_filtered['S_Domain']=='Khaishi']

SiO2,Na2O,K2O = smp_data[['SiO2','Na2O','K2O']].T.values
gp.TASsm(SiO2,Na2O,K2O)

fig = plt.figure()
ax = fig.add_subplot(111,projection='ternary')

fig2,axs = plt.subplots(2,figsize=(5,7))
fig3,ax3 = plt.subplots(1)

smp_groups = smp_data.groupby(smp_data.index)

for k,group in smp_groups:
    Tb,Th,Ta = group[['Tb','Th','Ta']].T.values
    gp.cabanis(Tb,Th,Ta,ax=ax,label=k)
    gp.REE(group,ax=axs[0])
    gp.immobile(group,ax=axs[1])
    ax3.scatter(group['Longitude'],group['Latitude'])

ax.legend()




In [ ]:
# plot gtoids on whalen plot

fig, axs = plt.subplots(2)

axs[0].set_xlim(3,100)
axs[0].set_ylim(0.08,2)
axs[0].set_xscale('log')
axs[0].set_yscale('log')

axs[0].vlines(60,ymin=0.01,ymax=10)
axs[0].hlines(y=0.4,xmin=1,xmax=60)

print(data_gtoid.index)

groups = data_gtoid.groupby('S_Domain')

for k,group in groups:
    nbyb = group['Nb']+group['Yb']
    nby = group['Nb']/group['Y']

    axs[0].scatter(nbyb,nby,label=k)

    fenumb = group['FeO*']/(group['FeO*']+group['MgO'])
    si = group['SiO2']

    axs[1].scatter(si,fenumb,label=k)

    #ax.annotate(group.index[0][-3:],(x,y))

axs[0].legend()